# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1273


## 讀取資料

In [2]:
train_X, train_y = SVHN2.traindata(Float32, 1:20000)
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [3]:
train_X

32×32×3×20000 Array{Float32,4}:
[:, :, 1, 1] =
 0.129412   0.109804  0.156863   0.188235   …  0.337255   0.329412   0.333333
 0.0588235  0.054902  0.0705882  0.0745098     0.341176   0.337255   0.34902 
 0.0588235  0.054902  0.0627451  0.0901961     0.321569   0.321569   0.333333
 0.164706   0.168627  0.117647   0.0901961     0.313726   0.305882   0.333333
 0.262745   0.270588  0.25098    0.231373      0.243137   0.317647   0.317647
 0.266667   0.231373  0.227451   0.262745   …  0.0823529  0.180392   0.282353
 0.266667   0.231373  0.243137   0.239216      0.0784314  0.0901961  0.121569
 0.290196   0.298039  0.301961   0.305882      0.341176   0.352941   0.352941
 0.305882   0.32549   0.329412   0.34902       0.419608   0.427451   0.439216
 0.345098   0.345098  0.34902    0.352941      0.411765   0.403922   0.415686
 0.329412   0.321569  0.32549    0.329412   …  0.415686   0.4        0.396078
 0.309804   0.329412  0.32549    0.333333      0.411765   0.407843   0.423529
 0.309804   0.329

In [4]:
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×2000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  1  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  1  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [5]:
train_X = reshape(train_X, 32, 32, 3, :)
test_X = reshape(test_X, 32, 32, 3, :)
# train_y = onehotbatch(train_y, 0:9)
# test_y = onehotbatch(test_y, 0:9)

32×32×3×2000 Array{Float32,4}:
[:, :, 1, 1] =
 0.14902   0.152941  0.14902   0.14902    …  0.203922   0.196078   0.188235 
 0.152941  0.152941  0.145098  0.14902       0.207843   0.2        0.188235 
 0.152941  0.152941  0.152941  0.152941      0.2        0.192157   0.180392 
 0.160784  0.156863  0.156863  0.152941      0.172549   0.168627   0.152941 
 0.168627  0.160784  0.160784  0.160784      0.14902    0.145098   0.141176 
 0.172549  0.168627  0.168627  0.168627   …  0.133333   0.109804   0.0980392
 0.176471  0.176471  0.180392  0.156863      0.117647   0.0941176  0.0784314
 0.184314  0.184314  0.188235  0.137255      0.105882   0.0823529  0.0627451
 0.184314  0.184314  0.188235  0.129412      0.0941176  0.0705882  0.0470588
 0.184314  0.184314  0.196078  0.145098      0.0901961  0.0666667  0.0431373
 0.184314  0.188235  0.2       0.160784   …  0.0941176  0.0705882  0.054902 
 0.184314  0.192157  0.196078  0.172549      0.0823529  0.0705882  0.0627451
 0.184314  0.188235  0.196078 

In [6]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

DataLoader((Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627

## CNN 模型

In [7]:
# model = # write your model here
model = Chain(
    Conv((3, 3), 3=>32, relu),
    MaxPool((2, 2), pad=(0, 0, 0, 0), stride=(2, 2)),
    Conv((3, 3), 32=>32, relu),
    MaxPool((2, 2), pad=(0, 0, 0, 0), stride=(2, 2)),
    Conv((3, 3), 32=>64, relu),
    MaxPool((2, 2), pad=(0, 0, 0, 0), stride=(2, 2)),
    flatten, 
    Dense(1024, 256, relu),
    Dense(256, 10),
    softmax
)

Chain(Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(1024, 256, relu), Dense(256, 10), softmax)

In [8]:
using CuArrays
model = model |> gpu
train_X = train_X |> gpu
train_y = train_y |> gpu
test_X = test_X |> gpu
test_y = test_y |> gpu

10×2000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  1  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  1  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [9]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

In [10]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

test_loss (generic function with 1 method)

In [11]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

In [12]:
epochs = 20
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\aband\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


DimensionMismatch: DimensionMismatch("A has dimensions (256,1024) but B has dimensions (256,1024)")

In [13]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [14]:
accuracy(test_X, test_y)

DimensionMismatch: DimensionMismatch("A has dimensions (256,1024) but B has dimensions (256,2000)")